### 初期設定

In [1]:
# --- Notebook初期設定 ---
%load_ext autoreload
%autoreload 2
from utils.data_loarder import load_csv_to_db, get_total_assets
print("🔁 autoreload 有効化完了")
import os
import pandas as pd
import plotly.express as px

# 設定読み込み
#settings = cfg.load_settings("setting.yaml")
#display(settings)

🔁 autoreload 有効化完了


### 動作確認

In [2]:
csv_path = "./data/raw/資産詳細.csv"
load_csv_to_db(csv_path)


In [ ]:
# CSV読み込み → DB保存

# 総資産推移を取得
pivot = get_total_assets()
df = pivot.sum(axis=1)
df

# グラフ表示
fig = px.line(df, x=df.index, y=df.values, title="資産推移")
fig.show()